# Plotting Slab2 Regions:

This notebook should be used for creating 2d and 3d plots of Slab2 Regions. The pygmt conda environment must be installed, and Jupyter must be launched from this environment. Grid files, trench data, clipping data, CPT files, and tilted region data are all pulled from the internet and written locally. Please note that these files can be large, and may be deleted. See web_files.py for more information.

To use this notebook, run the first two cells, then read the Grid Track section or Plotting section depending on your needs

In [ ]:
import copy
import io
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import pygmt 
import requests
import shutil
from notebook_funcs import *
from pygmt.clib import Session
from web_files import *

In [ ]:
# Defining grd types in dictionary for ease of use and access
grd_dic = {
    'dep' : 'Depth',
    'dip' : "Dip",
    'thk' : 'Thinkness',
    'str' : 'Strike',
    'unc' : "Unc"
}
# Defining all slab regions
all_slabs = ['cal', 'cam', 'cot','hin', 'man', 'sco', 'sul', 'sam', 'cas', 'him', 
'puy', 'mak', 'hal', 'kur', 'mue', 'alu', 'ryu', 'phi', 'ker',
 'van', 'png', 'car', 'hel', 'pam', 'sol', 'sum', 'izu']
# Defining slabs with no trench
no_trench = ['hal', 'hin', 'pam']
# Slabs with a complex geomentry
tilted_slabs = ['izu', 'ker', 'man', 'sol']
# Slabs with clipping issues
map_issue = ['car', 'mue', 'sco', 'sam']

# Grid Track

To use the grid tracking function, run next two cells below. When prompted, enter the values for the slab, longitude and latitude.

In [ ]:
def grid_track() -> None:
    '''
    Function to print the 'z' value of a grid file at a given lon/lat position
    
    Parameters
    ----------
    None
    
    Returns
    --------
    None
    '''
    slab = input('Please Provide the 3-letter slab code : ')
    # Defining directory paths
    path_list = paths(slab, 'map')
    date = path_list[0]
    path = path_list[1]
    plot_dir = path_list[2]
    out_dir = path_list[3]
    # Defining file paths
    file_paths = grid_links(path,date,slab)
    files = file_paths[0]
    clp = file_paths[1]
    tilted_path = file_paths[2]
    # User input lon/lat
    lon = float(input('Please input the longitude value in positive or negative degrees East : '))
    lat = float(input('Please input the latitude value : '))
    loc = [lon,lat]
    for file in files:
        print('~'*35)  
        # grdinfo for each grid
        info = get_info(file, slab)
        region = info[0]
        # Ensuring lon values are in positive degrees East
        if loc[0] < 0:
            loc[0] += 360
        # Defining data frame of the location in question
        pos = pd.DataFrame([loc],columns = ['longitude', 'latitude'])
        # Collecting z value at position
        track = pygmt.grdtrack(
            points = pos,
            grid = file,
            newcolname = 'z'
        )
        grd_type = file[-16:-13]
        print(grd_type)
        print(track)
        
        # collecting z value if slab has complex geometry and location has z value in tilted region
        if slab in tilted_slabs:        
            index_dict = {
                'dep' : 2,
                'str' : 3,
                'dip' : 4,
                'unc' : 5,
                'thk' : 8
            }
            
            
            tilted = pd.read_csv(tilted_path)        
            # Changing to degrees East
            tilted.iloc[:,2] *= -1
            # Collecting relevant tilted data
            tilted = tilted.iloc[:,[0,1,index_dict[grd_type]]]
            # Chaning tilted data to grd file
            grid2 = pygmt.xyz2grd(
                    data = tilted,
                    region = region,
                    spacing = 0.05)
            
            # Tracking tilted grid file at position
            track2 = pygmt.grdtrack(
                points = pos,
                grid = grid2,
                newcolname = 'z')
            
            print(track2)
    print('~'*35)  

In [ ]:
grid_track()

# Plotting

The rest of the cells must be run in order to make plots.

The parameters below may be changed to fit the needs of the user.

When the cell is running, the user will be prompted for 3 inputs
1. To choose the slab region
2. Specify, True or False, to use a color blind friendly color palette (Only for 2d Plots)
3. plot_type controls what type of plot to make. '3d' will make a 3d plot, while anything else will make a 2d plot.

Optionally, the following parameters may be changed
* input_list will plot specified input data on the 3d plot. Specify a list of a string, representing the data type found in Slab2 input files (EQ, TO, BA, etc.), and a string representing the input database date to use, in the form MM-YY. The available input dates are 04-18, 12-19, 11-20, and 09-21.
* The projection of 2d plots may be changed, refer to https://www.pygmt.org/latest/projections/index.html?highlight=projection for different types of available projections
* The perspective of 2d plots may be changed. Refer to https://docs.generic-mapping-tools.org/latest/gmt.html#perspective-full for documentation

Once all of the parameters are specified, run the rest of the cells. 

2d plots will be stored locally, while 3d plots and grid tracking info will be in the output cell.

In [ ]:
slab = input('Please enter a slab : ')
color_blind = input('Please specify True to use color blind friendly colors, or False for normal coloring : ')
plot_type = input('Please specify 2d for 2D plots or 3d for an interactive 3D plot : ')
input_list = ['EQ', '04-18']
projection = 'M15c'
perspective = [180,90]

path_list = paths(slab, 'map')
date = path_list[0]
path = path_list[1]
plot_dir = path_list[2]
out_dir = path_list[3]

In [ ]:
def plot(file : str, region : list, z: list, clp : str, tilted: pd.DataFrame) -> None:
    """
    Funtion to create 2d plots of infomation stored in grd files.
    This function will populate a directory within a Slab2 Output directory 
    with pdf files of the plot type requested
    Parameters
    ----------
    file : str
        path to a grd file to plot
    region : list of (float/int, float/int)
        The region of interest to plot
    z : list of (float, float)
        The minimum and maximum values in the grd file
    clp : str
        The link to the clip file
    tilted : pd.DataFrame    
        The tilted portion of the slab if slab has complex geometry
        None if slab does not have a tilted region
    Returns
    ----------
    None
    """
    # Collecting files used in plotting
    trench = get_trench()
    ghayes_cpt = get_ghayes(plot_dir)
    bath = get_bath(plot_dir)
    
    # Defining the type of grid file
    grd_type = file[-16:-13]
    
    # Reading clipping file    
    clp = pd.read_csv(clp)
    # Clipping file
    if slab not in map_issue:
        file = clip_grd(file, clp, region, grd_type, date, slab, projection)
    # Selecting relevant trench data
    trenchdf = trench
    trenchdf = trenchdf.loc[trenchdf.slab == slab]
    trenchdf = trenchdf.iloc[:,[0,1]]
    
    # Defining expended grid type (e.g. Thickness, Depth, etc.)
    grd = grd_dic[grd_type]
    name = f'{grd} of {slab} {date}'
    # Name of file to stoe pdf in
    fname = f'{out_dir}/{slab}_{date}_{grd_type}.pdf'
    print(f'Creating plot for {name}')
    
    # Begin Plotting
    fig = pygmt.Figure()

    #Plotting world topography below
    fig.grdimage(
                grid = bath, 
                nan_transparent = True, 
                region = region,
                perspective = perspective,
                projection = projection,
                cmap = ghayes_cpt
                )
    
    #Creating coast outlines
    fig.coast(
        region = region, 
        shorelines = True,
        projection = projection,            
        frame = ['ag', f'+t"{name}"'],
        perspective = perspective,
        )
    # Defining color palettes and units
    cmap_dict = {
        'dep' : ['seis','Km'],
        'str' : ['no_green','Deg'],
        'dip' : ['no_green','Deg'],
        'thk' : ['no_green','Km'],
        'unc' : ['gray']
    }
    # Creating cpt below
    if color_blind == True:
        cmap = get_colorblind(plot_dir)
        
    else:
        cmap = cmap_dict[grd_type][0]
    if z[0] == z[1]:
        pass
    else:

        pygmt.grd2cpt(
            grid = file,
            cmap = cmap,
            reverse = True,
            continuous = True,
            output = 'cpt.cpt')
        
    
    # Plotting the grd below with the cpt defined above
    fig.grdimage(
                grid = file, 
                nan_transparent = True, 
                region = region,
                transparency = 20,
                perspective = perspective,
                projection = projection,
                cmap = 'cpt.cpt'
                )
    
    # Creating cpts and plotting the areas of complex geometries
    if slab in tilted_slabs:
        index_dict = {
            'dep' : 2,
            'str' : 3,
            'dip' : 4,
            'unc' : 5,
            'thk' : 8
        }

        df = pd.read_csv(tilted)   

        df = df.iloc[:,[0,1,index_dict[grd_type]]] # Collecting relavent supplementary information
        
        if grd_type == 'dep':
            if slab == 'man':
                tilt_max = max(df.iloc[:,2])
                z_coloring = f'{str(z[0])}/{str(tilt_max)}/10'
            else:
                z_coloring = f'{str(-1*z[1])}/{str(-1*z[0])}/10'
            cpt = pygmt.makecpt(
            cmap = cmap,
            series = z_coloring,
            background = True,
            color_model = 'r',
            continuous = True,
            output = 'cpt2.cpt'
            )
            # Plotting complex dep surface
            fig.plot(
                data = df,
                projection = projection,
                region = region,
                style = 'c0.08',
                cmap = 'cpt2.cpt' 
            )
            
        else:
            # Plotting other grd type surfaces
            fig.plot(
                data = df,
                projection = projection,
                region = region,
                style = 'c0.08',
                cmap = 'cpt.cpt'
            )
        
    # Drawing contours
    fig.grdcontour(
        grid = file,
        perspective = perspective,
        projection = projection
    )
    
    #Plotting Slab outline
    fig.plot(
        data = clp,
        pen = '.5,black',
        region = region,
        projection = projection
    )

    #Plotting Trench
    if slab not in no_trench:
        fig.plot(
            data = trenchdf,
            projection = projection,
            pen = '2,yellow',
            label = 'Trench'
        )
        fig.legend(
        box = True
        )
    # Creating colorbar
    if grd_type != 'unc':
        frame = [f"x+l{grd} ({cmap_dict[grd_type][1]})"]
    else:
        frame = [f"x+l{grd}"]
    fig.colorbar(
        cmap = 'cpt.cpt',
        frame = frame,
        ) 
    
    # Saves figure 
    print(f'Saving Plot in : {fname}')
    print('-'*40)
    fig.savefig(fname = fname)
    fig.show()

In [ ]:
def plot_3d(file: str, region: list, z: list, tilted = None) -> None:
    """
    Function to create an interative 3d plot of a grid file
    
    Parameters
    -----------
    file : str
        Path to the grid file
    region : list of (float/int, float/int)
        The region of interest to plot
    z : list of (float, float)
        The minimum and maximum values in the grd file
    tilted : str
        Path to the tilted data file for tilted regions
        Defualt if None if slab is not complex
    
    Returns
    ---------
    None
    """
    # Defining type of grid file
    grd_type = file[-16:-13]
    grd = grd_dic[grd_type]
    # Path to clip file
    clip =f'{path}/{slab}_slab2_clp_{date}.csv' 
    clp = pd.read_csv(clip)
    # Clipping grid
    if slab not in map_issue:
        file = clip_grd(file, clp, region, grd_type, date, slab, projection)
        
    name = f'{grd} of {slab} {date}'
    
    print(f'Creating 3D plot for {name}')
    
    # Changing grid file to xyz
    data = pygmt.grd2xyz(grid = file)
    
    if slab in tilted_slabs:
        index_dict = {
            'dep' : 2,
            'str' : 3,
            'dip' : 4,
            'unc' : 5,
            'thk' : 8}
        # Collecting relevant tilted region data
        df = pd.read_csv(tilted)        
        df = df.iloc[:,[0,1,index_dict[grd_type]]]
        # Renaming Columns
        mapping = {df.columns[0]: 'x', df.columns[1]: 'y', df.columns[2]: 'z'}
        df.rename(columns = mapping, inplace=True)
        # Changing to degrees East
        for i in range(len(df)):
            df.iloc[i,2] *= -1
        # Appending tilted region to data to plot
        data = pd.concat([data,df]).reset_index(drop = True)   
        
    # Scatter plot of data
    fig = px.scatter_3d(data, x = 'x', y = 'y', z = 'z', color = 'z')
    # Plotting input data if specified
    if input_list != None:
        data_type = input_list[0]
        database_date = input_list[1]
        # Collecting input data
        input_df = map_input_data(data_type, database_date, slab)
        if data_type == 'EQ':
            size = input_df.mag
        else:
            size = np.full(len(input_df),10)
        # Plotting input data
        fig.add_trace(go.Scatter3d(x = input_df.lon, 
                                   y = input_df.lat, 
                                   z = input_df.depth,
                                   mode = 'markers',
                                   text = input_df.mag,
                                   name = data_type,
                                   marker = dict(size = size, color = 'green')))
    fig.show()

In [ ]:
def main():  
    file_paths = grid_links(path,date,slab)
    files = file_paths[0]
    clp = file_paths[1]
    tilted = file_paths[2]
    for file in files:
        info = get_info(file, slab)
        region = info[0]
        z = info[1]
        if plot_type == '2d':            
            plot(file, region, z, clp, tilted)
            
        else:
            if 'dep' in file:
                plot_3d(file, region, z, tilted)
            else:
                pass
            
    if os.path.exists('cpt.cpt'):
        os.remove('cpt.cpt')
        
    if os.path.exists('cpt2.cpt'):
        os.remove('cpt2.cpt')

In [ ]:
if __name__ == '__main__':
    main()